In [21]:
%%writefile mergesort.cu

// Include necessary standard libraries
#include <iostream>
#include <vector>
#include <cmath>
#include <chrono>
#include <algorithm>

using namespace std;
using namespace std::chrono;

// Calculate Euclidean distance between two points
double calculateDistance(const float2 &p1, const float2 &p2) {
    return sqrt((p1.x - p2.x) * (p1.x - p2.x) + (p1.y - p2.y) * (p1.y - p2.y));
}

// Perform mergesort on a slice of the array
void merge(vector<float>& array, int const left, int const mid, int const right) {
    auto const subArrayOne = mid - left + 1;
    auto const subArrayTwo = right - mid;
  
    vector<float> leftArray(subArrayOne), rightArray(subArrayTwo);
  
    for (auto i = 0; i < subArrayOne; i++)
        leftArray[i] = array[left + i];
    for (auto j = 0; j < subArrayTwo; j++)
        rightArray[j] = array[mid + 1 + j];
  
    int indexOfSubArrayOne = 0, indexOfSubArrayTwo = 0;
    int indexOfMergedArray = left;
  
    while (indexOfSubArrayOne < subArrayOne && indexOfSubArrayTwo < subArrayTwo) {
        if (leftArray[indexOfSubArrayOne] <= rightArray[indexOfSubArrayTwo]) {
            array[indexOfMergedArray] = leftArray[indexOfSubArrayOne];
            indexOfSubArrayOne++;
        } else {
            array[indexOfMergedArray] = rightArray[indexOfSubArrayTwo];
            indexOfSubArrayTwo++;
        }
        indexOfMergedArray++;
    }

    while (indexOfSubArrayOne < subArrayOne) {
        array[indexOfMergedArray] = leftArray[indexOfSubArrayOne];
        indexOfSubArrayOne++;
        indexOfMergedArray++;
    }
    while (indexOfSubArrayTwo < subArrayTwo) {
        array[indexOfMergedArray] = rightArray[indexOfSubArrayTwo];
        indexOfSubArrayTwo++;
        indexOfMergedArray++;
    }
}

// Recursively perform mergesort
void mergeSort(vector<float>& array, int const begin, int const end) {
    if (begin >= end)
        return;

    auto mid = begin + (end - begin) / 2;
    mergeSort(array, begin, mid);
    mergeSort(array, mid + 1, end);
    merge(array, begin, mid, end);
}

int main() {    
    // Create n points uniformly in [100, 100] grid
    int n = 16;
    vector<float2> h_points(n);
    float side = sqrt(n);  // Number of points along one side of the grid
    float step_x = 100.0 / (side - 1);  // Step size for x coordinate
    float step_y = 100.0 / (side - 1);  // Step size for y coordinate
    for (int i = 0; i < side; i++) {
        for (int j = 0; j < side; j++) {
            int index = i * side + j;
            h_points[index].x = j * step_x;
            h_points[index].y = i * step_y;
        }
    }
    float2 qrand = {5, 5}; // Query point
    vector<float> distances(n);
    
    
    
    // Uncomment out to run k-Nearest Neighbors once
    cout << "-----POINTS-----" << endl;
    for (int i = 0; i < n; i++) {
        cout << "[" << h_points[i].x << ", " << h_points[i].y << "]" << endl;
    }
    
    auto start_time = high_resolution_clock::now();
    
    for (int i = 0; i < n; i++) {
        distances[i] = calculateDistance(h_points[i], qrand);
    }
    
    cout << "-----BEFORE-----" << endl;
    for (int i = 0; i < n; i++) {
        cout << i << ": " << distances[i] << endl;
    }
    
    mergeSort(distances, 0, n - 1);
        
    cout << "-----AFTER-----" << endl;
    for (int i = 0; i < n; i++) {
        cout << i << ": " << distances[i] << endl;
    }
    
    // Note: execution times longer if distances are outputted
    auto end_time = high_resolution_clock::now();
    auto duration = duration_cast<microseconds>(end_time - start_time);
    cout << "Execution time: " << duration.count() << " microseconds" << endl;
    
    
    
    // Uncomment out to get execution time of n trials
    /*
    microseconds totalDuration(0);
    
    int n_trials = 100;
    cout << "Computing k-Nearest Neighbors: " << n_trials << " times" << endl;

    for (int i = 0; i < n_trials; ++i) {
        auto start_time = high_resolution_clock::now();

        // Fine the nearest neighbors
        for (int i = 0; i < n; i++) {
            distances[i] = calculateDistance(h_points[i], qrand);
        }

        mergeSort(distances, 0, n - 1);

        auto end_time = high_resolution_clock::now();
        auto duration = duration_cast<microseconds>(end_time - start_time);
        totalDuration += duration;
        
        cout << ".";
    }
    cout << endl;

    // Calculate average execution time
    auto averageDuration = totalDuration.count() / n_trials;
    cout << "Average execution time: " << averageDuration << " microseconds" << endl;
    */
    
    return 0;
}

Overwriting mergesort.cu


In [22]:
!nvcc -o mergesort mergesort.cu
!./mergesort

-----POINTS-----
[0, 0]
[33.3333, 0]
[66.6667, 0]
[100, 0]
[0, 33.3333]
[33.3333, 33.3333]
[66.6667, 33.3333]
[100, 33.3333]
[0, 66.6667]
[33.3333, 66.6667]
[66.6667, 66.6667]
[100, 66.6667]
[0, 100]
[33.3333, 100]
[66.6667, 100]
[100, 100]
-----BEFORE-----
0: 7.07107
1: 28.7711
2: 61.869
3: 95.1315
4: 28.7711
5: 40.0694
6: 67.8642
7: 99.1351
8: 61.869
9: 67.8642
10: 87.2098
11: 113.26
12: 95.1315
13: 99.1351
14: 113.26
15: 134.35
-----AFTER-----
0: 7.07107
1: 28.7711
2: 28.7711
3: 40.0694
4: 61.869
5: 61.869
6: 67.8642
7: 67.8642
8: 87.2098
9: 95.1315
10: 95.1315
11: 99.1351
12: 99.1351
13: 113.26
14: 113.26
15: 134.35
Execution time: 123 microseconds
